In [1]:
from google import genai

client = genai.Client()

MODEL_NAME = "gemini-2.0-flash"
DISPLAY_NAME = "dynamic-prompt-gen-job"

In [2]:
def poll_job_status(client, job_name, interval=30):
    """Poll the status of a batch job until it finishes."""
    import time
    while True:
        batch_job = client.batches.get(name=job_name)
        if batch_job.state.name in ('JOB_STATE_SUCCEEDED', 'JOB_STATE_FAILED', 'JOB_STATE_CANCELLED', 'JOB_STATE_EXPIRED'):
            print(f"Job finished with state: {batch_job.state.name}")
            return batch_job
        print(f"Job not finished. Current state: {batch_job.state.name}. Waiting {interval} seconds...")
        time.sleep(interval)
        
def create_inline_requests(book_review_prompts):
    """Generate inline requests for a list of book reviews."""
    inline_requests = []
    for book_review_prompt in book_review_prompts:
        request_obj = {"contents": [{"parts": [{"text": book_review_prompt}], "role": "user"}]}
        inline_requests.append(request_obj)
    return inline_requests

In [13]:
def create_system_instruction():
    return """
Bạn là một nhà phân tích văn học chuyên nghiệp, có khả năng thích ứng với nhiều thể loại và yêu cầu khác nhau. Dựa vào `book_title` và `main_topic`.

**YÊU CẦU ĐẦU RA:**
- Kết quả phải tuân thủ các **Yêu cầu Bổ sung** nếu được cung cấp.
- Không cần thêm câu từ mang tính giao tiếp. Không trả về JSON.

**QUY TRÌNH CỦA BẠN GỒM HAI BƯỚC:**

**1. Kế hoạch & Cấu trúc (Dàn ý):**
- **Phân tích yêu cầu:** Dựa trên thể loại sách và chủ đề, hãy xác định cấu trúc dàn ý logic và phù hợp nhất.
- **Tạo dàn ý:** Xây dựng một dàn ý chi tiết. Dàn ý phải toàn diện, bao quát các khía cạnh chính của chủ đề. Mỗi đề mục chính cần được làm rõ bằng các gạch đầu dòng con để đi sâu vào chi tiết.

**2. Thu thập & Tóm tắt:**
- Dựa trên dàn ý đã tạo, viết một đoạn tóm tắt ngắn cho MỖI đề mục chính, cung cấp thông tin cốt lõi và súc tích.

**NGUYÊN TẮC HƯỚNG DẪN (ADAPTIVE GUIDELINES):**
- **Văn phong (tone):** Nếu trong `state` có khóa 'tone', hãy điều chỉnh giọng văn cho phù hợp (ví dụ: 'học thuật', 'thân mật', 'phê bình sâu sắc'). Nếu không, hãy dùng giọng văn phân tích trung lập.
- **Điểm nhấn (focus_points):** Nếu trong `state` có khóa 'focus_points' (là một danh sách), hãy đảm bảo dàn ý và phần tóm tắt tập trung làm nổi bật những điểm này.
- **Độ sâu (depth):** Nếu trong `state` có khóa 'depth' (ví dụ: 'sơ lược' hoặc 'chi tiết'), hãy điều chỉnh số lượng đề mục và mức độ chi tiết của thông tin cho phù hợp.
- **Ngôn ngữ**: Dù đầu vào có thể là tiếng Anh, bạn CHỈ ĐƯỢC trả lời bằng **tiếng Việt**.
"""

In [14]:
import csv

request_messages = []

# Đọc file JSON 400 cuốn sách
with open("book_reviews_prompts.csv", "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)  # Sử dụng DictReader để đọc từng dòng dưới dạng dict
    for row in reader:
        request_messages.append(row["response"])

request_messages

['Viết bài đánh giá về "African Cooking 101" tập trung vào cách cuốn sách giới thiệu sự pha trộn và ảnh hưởng đa dạng của ẩm thực châu Phi, vượt qua những định kiến chung về "món ăn châu Phi" là gì.',
 'Viết bài đánh giá về "The Collected Poems of Dylan Thomas" tập trung vào cách thơ ca của ông khám phá sự giằng xé giữa khao khát sự bất tử và chấp nhận sự hữu hạn của cuộc sống, đặc biệt khi nhìn vào bối cảnh cái chết sớm của nhà thơ.',
 'Đánh giá cuốn "Tisarian\'s Treasure" tập trung vào vai trò của ma thuật cổ xưa và sức mạnh tiềm ẩn của nó trong việc định hình số phận của các nhân vật, đặc biệt là mối liên hệ giữa kho báu và những sinh vật quái dị trong rừng rậm.',
 'Đánh giá "The Crazy Iris and Other Stories of the Atomic Aftermath" tập trung vào sự giằng xé giữa hy vọng và tuyệt vọng trong cuộc sống của những người sống sót sau thảm họa hạt nhân.',
 'Tôi muốn viết bài đánh giá về tập thơ "Words for Relocation" tập trung vào cách tác giả sử dụng ngôn ngữ để khám phá những giằng xé v

In [10]:
from google.genai import types

system_instruction=create_system_instruction()

with open("research_book_reviews_response.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["request_message", "response"])  # Write header

    # Loop through all request messages
    for request_message in request_messages:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=request_message,
            config=types.GenerateContentConfig(
                system_instruction=system_instruction
            ),
        )
        # Write the request and response to the CSV file
        writer.writerow([request_message, response.text])
        
print("Responses saved to 'research_book_reviews_response.csv'")

Responses saved to 'research_book_reviews_response.csv'


In [15]:
from concurrent.futures import ThreadPoolExecutor
from google.genai import types
import csv

def process_requests_chunk(request_chunk, system_instruction):
    """Process a chunk of requests and return the results."""
    results = []
    for request_message in request_chunk:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=request_message,
            config=types.GenerateContentConfig(
                system_instruction=system_instruction
            ),
        )
        results.append((request_message, response.text if response and response.text else "Error: No response"))
    return results

# Create system instruction
system_instruction = create_system_instruction()

# Split the request messages into chunks of 10
chunk_size = 10
request_chunks = [request_messages[i:i + chunk_size] for i in range(0, len(request_messages), chunk_size)]

# Open the CSV file to save responses
with open("research_book_reviews_response.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["No.", "request_message", "response"])  # Write header

    # Use ThreadPoolExecutor to process chunks in parallel
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_requests_chunk, chunk, system_instruction) for chunk in request_chunks]

        # Collect results as they complete
        no = 1
        for future in futures:
            for request_message, response_text in future.result():
                writer.writerow([no, request_message, response_text])
                no += 1

print("Responses saved to 'research_book_reviews_response_quick.csv'")

Responses saved to 'research_book_reviews_response_quick.csv'
